In the cell below, flight data is retrieved from the OpenSky Network API using the `requests` library. The JSON response is parsed, and each flight's state vector is converted into a dictionary with named fields (such as `icao24`, `callsign`, `origin_country`, etc.). These dictionaries are collected into a list called `flights`. Finally, a pandas DataFrame named `df` is created from the raw API data and its first few rows are printed for inspection.

In [ ]:
import pandas as pd
import requests

url = "https://opensky-network.org/api/states/all"
response = requests.get(url)

data = response.json()

# Convert to list of dictionaries for convenience
flights = []
for state in data.get("states", []):
    state = state + [None]*(18 - len(state))

    flights.append({
        "icao24": state[0],
        "callsign": state[1],
        "origin_country": state[2],
        "time_position": state[3],
        "last_contact": state[4],
        "longitude": state[5],
        "latitude": state[6],
        "baro_altitude": state[7],
        "on_ground": state[8],
        "velocity": state[9],
        "true_track": state[10],
        "vertical_rate": state[11],
        "sensors": state[12],
        "geo_altitude": state[13],
        "squawk": state[14],
        "spi": state[15],
        "position_source": state[16],
        "category": state[17]
    })

# Create DataFrame
df = pd.DataFrame(data)

print(df.head())

         time                                             states
0  1756425371  [ab1644, UAL2116 , United States, 1756425370, ...
1  1756425371  [e8027b, LAN708  , Chile, 1756425329, 17564253...
2  1756425371  [aa3cbe, N759PA  , United States, 1756425359, ...
3  1756425371  [801638, AXB1197 , India, 1756425371, 17564253...
4  1756425371  [aa9321, UAL57   , United States, 1756425094, ...


### Initiating EDA(Exploratory Data Analysis)

In [13]:
df.head()

,time,states
0,1756425371,"[ab1644, UAL2116 , United States, 1756425370, ..."
1,1756425371,"[e8027b, LAN708 , Chile, 1756425329, 17564253..."
2,1756425371,"[aa3cbe, N759PA , United States, 1756425359, ..."
3,1756425371,"[801638, AXB1197 , India, 1756425371, 17564253..."
4,1756425371,"[aa9321, UAL57 , United States, 1756425094, ..."


In [35]:

def pad_state(state):
    if state is None:
        return [None]*18
    return state + [None]*(18 - len(state))


states_padded = df['states'].apply(pad_state)

# Cria DataFrame expandido
df_expanded = pd.DataFrame(states_padded.tolist(), columns=[
    "icao24",
    "callsign",
    "origin_country",
    "time_position",
    "last_contact",
    "longitude",
    "latitude",
    "baro_altitude",
    "on_ground",
    "velocity",
    "true_track",
    "vertical_rate",
    "sensors",
    "geo_altitude",
    "squawk",
    "spi",
    "position_source",
    "category"
])

# Mantém a coluna 'time'
df_final = pd.concat([df['time'], df_expanded], axis=1)

df_final.head()

,time,icao24,callsign,origin_country,time_position,last_contact,longitude,latitude,baro_altitude,on_ground,velocity,true_track,vertical_rate,sensors,geo_altitude,squawk,spi,position_source,category
0,1756425371,ab1644,UAL2116,United States,1.756425e+09,1756425370,-92.1416,34.6841,10980.42,False,194.52,258.87,0.33,None,11605.26,0504,False,0,None
1,1756425371,e8027b,LAN708,Chile,1.756425e+09,1756425329,-56.3436,-10.8792,11277.60,False,263.09,40.48,0.00,None,11978.64,None,False,0,None
2,1756425371,aa3cbe,N759PA,United States,1.756425e+09,1756425359,-111.9905,41.7878,2042.16,False,56.39,301.95,-0.65,None,2156.46,None,False,0,None
3,1756425371,801638,AXB1197,India,1.756425e+09,1756425371,79.3739,27.3752,10485.12,False,224.67,113.34,3.90,None,11170.92,None,False,0,None
4,1756425371,aa9321,UAL57,United States,1.756425e+09,1756425094,-62.1812,44.5083,11277.60,False,287.58,63.43,0.00,None,11551.92,None,False,0,None


In [36]:
df_final.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8274 entries, 0 to 8273
Data columns (total 19 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   time             8274 non-null   int64  
 1   icao24           8274 non-null   object 
 2   callsign         8274 non-null   object 
 3   origin_country   8274 non-null   object 
 4   time_position    8208 non-null   float64
 5   last_contact     8274 non-null   int64  
 6   longitude        8208 non-null   float64
 7   latitude         8208 non-null   float64
 8   baro_altitude    7614 non-null   float64
 9   on_ground        8274 non-null   bool   
 10  velocity         8273 non-null   float64
 11  true_track       8274 non-null   float64
 12  vertical_rate    7642 non-null   float64
 13  sensors          0 non-null      object 
 14  geo_altitude     7570 non-null   float64
 15  squawk           3482 non-null   object 
 16  spi              8274 non-null   bool   
 17  position_sourc

In [37]:
df_final.drop(['category', 'sensors'], axis=1, inplace=True)
df_final['time'] = pd.to_datetime(df_final['time'], unit='s')
df_final['time_position'] = pd.to_datetime(df_final['time'], unit='s')
df_final['last_contact'] = pd.to_datetime(df_final['last_contact'], unit='s')
df_final.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8274 entries, 0 to 8273
Data columns (total 17 columns):
 #   Column           Non-Null Count  Dtype         
---  ------           --------------  -----         
 0   time             8274 non-null   datetime64[ns]
 1   icao24           8274 non-null   object        
 2   callsign         8274 non-null   object        
 3   origin_country   8274 non-null   object        
 4   time_position    8274 non-null   datetime64[ns]
 5   last_contact     8274 non-null   datetime64[ns]
 6   longitude        8208 non-null   float64       
 7   latitude         8208 non-null   float64       
 8   baro_altitude    7614 non-null   float64       
 9   on_ground        8274 non-null   bool          
 10  velocity         8273 non-null   float64       
 11  true_track       8274 non-null   float64       
 12  vertical_rate    7642 non-null   float64       
 13  geo_altitude     7570 non-null   float64       
 14  squawk           3482 non-null   object 

In [38]:
df_final.head()

,time,icao24,callsign,origin_country,time_position,last_contact,longitude,latitude,baro_altitude,on_ground,velocity,true_track,vertical_rate,geo_altitude,squawk,spi,position_source
0,2025-08-28 23:56:11,ab1644,UAL2116,United States,2025-08-28 23:56:11,2025-08-28 23:56:10,-92.1416,34.6841,10980.42,False,194.52,258.87,0.33,11605.26,0504,False,0
1,2025-08-28 23:56:11,e8027b,LAN708,Chile,2025-08-28 23:56:11,2025-08-28 23:55:29,-56.3436,-10.8792,11277.60,False,263.09,40.48,0.00,11978.64,None,False,0
2,2025-08-28 23:56:11,aa3cbe,N759PA,United States,2025-08-28 23:56:11,2025-08-28 23:55:59,-111.9905,41.7878,2042.16,False,56.39,301.95,-0.65,2156.46,None,False,0
3,2025-08-28 23:56:11,801638,AXB1197,India,2025-08-28 23:56:11,2025-08-28 23:56:11,79.3739,27.3752,10485.12,False,224.67,113.34,3.90,11170.92,None,False,0
4,2025-08-28 23:56:11,aa9321,UAL57,United States,2025-08-28 23:56:11,2025-08-28 23:51:34,-62.1812,44.5083,11277.60,False,287.58,63.43,0.00,11551.92,None,False,0
